In [ ]:
# Importação de bibliotecas

import os
import sys
import pandas as pd
import polars as pl
from datasets import Dataset
import re
import spacy
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer, LabelEncoder
from bertopic import BERTopic
from transformers import BertTokenizer, BertModel, BertForSequenceClassification, Trainer, TrainingArguments, DataCollatorWithPadding
import torch
from torch import nn

In [ ]:
# Definicao da raiz do projeto

PROJECT_ROOT = 'G:/Csouza/nlp/topic_modeling'

os.chdir(PROJECT_ROOT)

sys.path.insert(0, PROJECT_ROOT)

In [ ]:
def extract(extract_path):
    csv_files = [file for file in os.listdir(extract_path) if file.endswith('.csv')]
    schema = pl.read_csv(f'{extract_path}/{csv_files[0]}', ignore_errors=True, separator=';').schema
                
    dfs = []

    for csv_file in csv_files:
        full_file_path = os.path.join(extract_path, csv_file) 
        df = pl.read_csv(full_file_path, ignore_errors=True, schema=schema, separator=';')
        dfs.append(df)

    return pl.concat(dfs)

def get_spacy_model(model='en_core_web_sm'):
    """
    Baixa o modelo de linguagem spaCy se não estiver presente.
    """
    try:
        nlp = spacy.load(model)
    except OSError:
        from spacy.cli import download
        download(model)
        nlp = spacy.load(model)
    return nlp

def preprocess_tokens(text, nlp_model=get_spacy_model(), special_char_remover=re.compile(r'[^A-Za-z\s]')):
    """
    Pré-processa o texto para uso em um modelo de PLN.

    Args:
    - text (str): Texto a ser pré-processado.
    - nlp_model (spacy.language.Language): Modelo spaCy para processamento de linguagem natural.

    Returns:
    - str: Texto pré-processado.
    """
    if not isinstance(text, str):
        raise ValueError("O argumento 'text' deve ser uma string.")
    
    text = special_char_remover.sub('', text)
    text = ' '.join(text.split())
    text = text.lower()

    doc = nlp_model(text)

    tokens = [token.lemma_ for token in doc if not token.is_stop and token.is_alpha]

    return tokens

In [ ]:
data_path = os.path.join(PROJECT_ROOT, 'data', 'internal', 'fapesp_projects')

data = extract(data_path)

variables = {
'N. Processo': 'n_processo',
'Data de Início': 'data',
'Título (Português)': 'titulo',
'Grande Área do Conhecimento': 'grande_area',
'Área do Conhecimento': 'area',
'Subárea do Conhecimento': 'subarea',
'Assuntos': 'assuntos',
'Resumo (Português)': 'resumo'}

data = (
    data
    .lazy()
    .rename(variables)
    .select(variables.values())
    .filter(
        pl.col('n_processo').is_not_null(),
        pl.col('resumo').is_not_null(),
        pl.col('resumo') != '')
    .with_columns(
        pl.col('data').str.to_datetime('%Y-%m-%d').dt.year().alias('ano'),
        pl.col('data').str.to_datetime('%Y-%m-%d').dt.month().alias('mes'))
    .select(pl.exclude('data'))
).collect()

data.head(3)